Author: Julia Wervers
Student Number: 13168665

This is the main script for the final project of the data processing course. The objective of the project is to find how representative lists of keywords extracted from summaries of books compared to keywords extracted from the book themselves. 

In [24]:
#Imports
import spacy
# Load a large nlp pipeline with vectors
nlp = spacy.load("en_core_web_lg")

In [31]:
###Function to get wordlist of a book

def get_book_wordlist(book_textfile):
    #Get text 
    f = open(book_textfile, "r", encoding="utf8")
    book_text = f.read()
    f.close()


    doc = nlp(book_text)
    #Iterate through the tokens and get the lowercase lemma of each token if the token is not a stopword and is not punctuation
    lemmas = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]

    #clean up the extracted text
    processed_text = " ".join(lemmas).strip()
    #Split the text up using space into a list of words
    book_wordlist = processed_text.split()
                      
    return(book_wordlist)

In [32]:
print(get_book_wordlist("Frankenstein.txt"))

['\ufeffletter', '1', 'mrs.', 'saville', 'england', 'st.', 'petersburgh', 'dec.', '11th', '17—.', 'rejoice', 'hear', 'disaster', 'accompany', 'commencement', 'enterprise', 'regard', 'evil', 'foreboding', 'arrive', 'yesterday', 'task', 'assure', 'dear', 'sister', 'welfare', 'increase', 'confidence', 'success', 'undertaking', 'far', 'north', 'london', 'walk', 'street', 'petersburgh', 'feel', 'cold', 'northern', 'breeze', 'play', 'cheek', 'brace', 'nerve', 'fill', 'delight', 'understand', 'feeling', 'breeze', 'travel', 'region', 'advance', 'give', 'foretaste', 'icy', 'clime', 'inspirited', 'wind', 'promise', 'daydream', 'fervent', 'vivid', 'try', 'vain', 'persuade', 'pole', 'seat', 'frost', 'desolation', 'present', 'imagination', 'region', 'beauty', 'delight', 'margaret', 'sun', 'visible', 'broad', 'disk', 'skirt', 'horizon', 'diffuse', 'perpetual', 'splendour', 'leave', 'sister', 'trust', 'precede', 'navigator', 'snow', 'frost', 'banish', 'sail', 'calm', 'sea', 'waft', 'land', 'surpass',